In [1]:
#Carson Batchelor

In [2]:
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load        Tokens  Owns (effective)  Host ID                               Rack 
UN  172.29.0.3  104.33 KiB  16      69.7%             b138ace0-45d7-4b47-9efe-dde10abc7bbc  rack1
UN  172.29.0.2  104.33 KiB  16      66.3%             afdb2570-0c26-48ce-b534-59dbd35dd143  rack1
UN  172.29.0.4  104.32 KiB  16      63.9%             e8659979-0ce6-4c7e-a02e-3beaaf4352d2  rack1



In [3]:
from cassandra.cluster import Cluster
cluster = Cluster(['p6-db-1', 'p6-db-2', 'p6-db-3'])
cass = cluster.connect()

In [4]:
cass.execute("DROP keyspace IF EXISTS weather")
cass.execute("""
    CREATE keyspace weather
    WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 3}
""")
cass.execute("USE weather")

In [5]:
cass.execute("DROP table IF EXISTS station")
cass.execute("DROP type IF EXISTS station_record")

cass.execute("""
    CREATE TYPE station_record (
        tmin int,
        tmax int)
""")

cass.execute("""
    CREATE TABLE stations (
        id text,
        name text STATIC,
        date date,
        record station_record,
        PRIMARY KEY (id, date)
    )
""")

In [6]:
#q1
cass.execute("describe table weather.stations").one().create_statement

"CREATE TABLE weather.stations (\n    id text,\n    date date,\n    name text static,\n    record station_record,\n    PRIMARY KEY (id, date)\n) WITH CLUSTERING ORDER BY (date ASC)\n    AND additional_write_policy = '99p'\n    AND bloom_filter_fp_chance = 0.01\n    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}\n    AND cdc = false\n    AND comment = ''\n    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}\n    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}\n    AND memtable = 'default'\n    AND crc_check_chance = 1.0\n    AND default_time_to_live = 0\n    AND extensions = {}\n    AND gc_grace_seconds = 864000\n    AND max_index_interval = 2048\n    AND memtable_flush_period_in_ms = 0\n    AND min_index_interval = 128\n    AND read_repair = 'BLOCKING'\n    AND speculative_retry = '99p';"

In [7]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .appName("p6")
         .config('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.4.0')
         .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions")
         .getOrCreate())

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d01b20ac-e7dc-4786-95af-fed77244e548;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found com.github.stephenc.jcip#jcip-annotations;1.0-1 in central
	found com.gith

In [8]:
df = spark.read.text("ghcnd-stations.txt")

In [9]:
from pyspark.sql.functions import col, expr

In [10]:
df2 = df.withColumn("station_ID", expr("substring(value, 0, 11)"))
df2 = df2.withColumn("state", expr("substring(value, 39, 2)"))
df2 = df2.withColumn("station_name", expr("substring(value, 42, 30)"))
df2

DataFrame[value: string, station_ID: string, state: string, station_name: string]

In [11]:
df_wisconsin = df2.filter(df2.state == "WI")

In [12]:
wi_stations = df_wisconsin.collect()
for row in wi_stations:
    station_id = row["station_ID"].strip()
    station_name = row["station_name"].strip()
    # Assuming you have a function to execute the INSERT statement, replace the following line accordingly
    cass.execute(
        """
        INSERT INTO weather.stations (id, name)
        VALUES (%s, %s)
        """,
        (station_id, station_name)
    )

In [13]:
import pandas as pd
pd.DataFrame(cass.execute("SELECT COUNT(*) FROM weather.stations"))

,count
0,1313


In [14]:
#pd.DataFrame(cass.execute("SELECT * FROM weather.stations"))

In [15]:
#q2
pd.DataFrame(cass.execute("""
    SELECT name 
    FROM weather.stations 
    WHERE id = 'USW00014837'
    """))["name"][0]

'MADISON DANE CO RGNL AP'

In [16]:
#q3
token = pd.DataFrame(cass.execute("""
    SELECT token(id) 
    FROM weather.stations 
    WHERE id = 'USC00470273'
    """))["system_token_id"][0]
token

-9014250178872933741

In [17]:
import subprocess
nodetool_output = subprocess.check_output(["nodetool", "ring"])

In [18]:
lines = nodetool_output.decode("utf-8").splitlines()
token_list = []
for line in lines:
    start = 80
    end = line.find(' ', start)
    token_list.append(line[80:end])

In [19]:
#q4
target_token = token
for token in token_list[5:53]:
    if int(token) >= int(target_token):
        smallest_token = str(token)
        break
    else:
        smallest_token = token_list[0]
int(smallest_token)

-8946087620239790516

In [20]:
# python3 -m grpc_tools.protoc -I=. --python_out=. --grpc_python_out=. station.proto

In [21]:
df = spark.read.parquet("records.parquet")

In [22]:
df_filter = df.filter((col("element").isin(["TMIN", "TMAX"])))
df_grouped = df_filter.groupBy("station", "date").pivot("element").agg(expr("first(value)"))

In [27]:
import grpc, station_pb2, station_pb2_grpc

channel = grpc.insecure_channel('localhost:5440')
client = station_pb2_grpc.StationStub(channel)

rows = df_grouped.collect()

for row in rows:
    year = row["date"][0:4]
    month = row["date"][4:6]
    day = row["date"][6:8]
    
    station_id = row["station"]
    date = str(f"{year}-{month}-{day}")
    tmin = int(row["TMIN"])
    tmax = int(row["TMAX"])

    response = client.RecordTemps(station_pb2.RecordTempsRequest(
        station=station_id,
        date=date,
        tmin=tmin,
        tmax=tmax
    ))

In [62]:
#q5
id = 'USW00014837'
response = client.StationMax(station_pb2.StationMaxRequest(station=id))
response.tmax

356

23/11/15 01:17:27 WARN ChannelPool: [s0|p6-db-2/172.29.0.3:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=a7a64868-12e6-4683-9374-88ab5f30ce2b, APPLICATION_NAME=Spark-Cassandra-Connector-local-1700010370528}): failed to send request (com.datastax.oss.driver.shaded.netty.channel.StacklessClosedChannelException))


In [29]:
stations_df = (spark.read.format("org.apache.spark.sql.cassandra")
    .option("spark.cassandra.connection.host", "p6-db-1,p6-db-2,p6-db-3")
    .option("keyspace", "weather")
    .option("table", "stations")
    .load())

stations_df.createOrReplaceTempView("stations")

In [30]:
#q6
spark.catalog.listTables()

[Table(name='stations', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [31]:
#q7
from pyspark.sql.functions import avg, col
results = spark.sql("""
    SELECT id, AVG(record.tmax - record.tmin) AS avg_dif
    FROM stations
    WHERE record IS NOT NULL
    GROUP BY id
""")

dict(results.collect())

{'USW00014837': 105.62739726027397,
 'USR0000WDDG': 102.06849315068493,
 'USW00014898': 102.93698630136986,
 'USW00014839': 89.6986301369863}

In [50]:
#q8
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load       Tokens  Owns (effective)  Host ID                               Rack 
DN  172.29.0.3  86.02 KiB  16      100.0%            b138ace0-45d7-4b47-9efe-dde10abc7bbc  rack1
UN  172.29.0.2  86.01 KiB  16      100.0%            afdb2570-0c26-48ce-b534-59dbd35dd143  rack1
UN  172.29.0.4  86.01 KiB  16      100.0%            e8659979-0ce6-4c7e-a02e-3beaaf4352d2  rack1



In [63]:
#q9
channel = grpc.insecure_channel('localhost:5440')
client = station_pb2_grpc.StationStub(channel)

id = 'USW00014839'
response = client.StationMax(station_pb2.StationMaxRequest(station=id))
response

error: "need 3 replicas, but only have 2"

In [64]:
#q10
station_id = 'USW00014837'
date = "2022-03-07"
tmin = 25
tmax = 130

response = client.RecordTemps(station_pb2.RecordTempsRequest(
    station=station_id,
    date=date,
    tmin=tmin,
    tmax=tmax))
response

23/11/15 01:18:27 WARN ChannelPool: [s0|p6-db-2/172.29.0.3:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=a7a64868-12e6-4683-9374-88ab5f30ce2b, APPLICATION_NAME=Spark-Cassandra-Connector-local-1700010370528}): failed to send request (com.datastax.oss.driver.shaded.netty.channel.StacklessClosedChannelException))
